In [1]:
import numpy as np

def preprocess(filename):
    ret = []
    with open(filename, 'r') as f:
        lines = f.readlines()
        for line in lines:
            line = line.lower()
            last_space = True
            new_line = []
            for i in line:
                if i.isalpha():
                    v = ord(i)-ord('a')+1
                    new_line += [[v]]
                    last_space = False
                else:
                    if last_space == False:
                        new_line += [[27]]
                        last_space = True
            ret += [new_line]
    return ret

jp_x = preprocess('jp.txt')
en_x = preprocess('en.txt')
x = jp_x + en_x
y = [[1.0,0.0] for i in jp_x] + [[0.0,1.0] for i in en_x]

In [2]:
print(len(x))
print(len(x[1]))

662
39


In [3]:
def align(data):
    max_len = 0
    for row in data:
        if len(row) > max_len:
            max_len = len(row)
    ret = []
    for row in data:
        ret += [row + [[0]]*(max_len-len(row))]
    return ret, max_len

x, seq_max_len = align(x)

In [4]:
print(len(x[0]))
print(seq_max_len)

75
75


In [5]:
import random
r_index = list(range(len(x)))
random.shuffle(r_index)
train_x = [x[i] for i in r_index[:int(len(r_index)*0.8)]]
train_y = [y[i] for i in r_index[:int(len(r_index)*0.8)]]
test_x = [x[i] for i in r_index[int(len(r_index)*0.8):]]
test_y = [y[i] for i in r_index[int(len(r_index)*0.8):]]

In [6]:
print(train_x[:2])
print(len(train_x))
print(len(train_y))
print(seq_max_len)
print(train_y[:10])

[[[7], [5], [14], [10], [9], [20], [19], [21], [27], [20], [15], [27], [25], [21], [13], [5], [27], [13], [1], [2], [15], [18], [15], [19], [8], [9], [27], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]], [[13], [9], [18], [1], [9], [27], [15], [27], [21], [18], [1], [7], [9], [20], [20], [1], [27], [14], [1], [18], [1], [27], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0]]]
529
529
75
[[1.0, 0.0], [1.0, 0.0], [1.0, 0.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [0.0, 1.0], [1.0, 0.0], [0.0, 1.0], [1.0, 0.0]]


In [7]:
import tensorflow as tf
print(tf.__version__)

/home/ziyi/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


1.4.0


In [15]:
n_hidden = 50
num_steps = 5000
n_classes = 2

def length(sequence):
    used = tf.sign(tf.reduce_max(tf.abs(sequence), reduction_indices=2))
    length = tf.reduce_sum(used, reduction_indices=1)
    length = tf.cast(length, tf.int32)
    return length

graph = tf.Graph()
with graph.as_default(), tf.device('/gpu:0'):
    
    weights = {
        'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([n_classes]))
    }
    
    
    def dynamicRNN(x, weights, biases):
        seqlen = length(x)
        x = tf.unstack(x, seq_max_len, 1)
        lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden)
        outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32,
                                    sequence_length=seqlen)
        print(len(outputs))
        outputs = tf.stack(outputs)
        print(outputs.shape)
        print(outputs[-1].shape)
        print(weights['out'].shape)
        print(biases['out'].shape)
       
        return tf.matmul(outputs[-1], weights['out']) + biases['out']
        
    x = tf.placeholder("float", [None, seq_max_len, 1])
    y = tf.placeholder("float", [None, n_classes])
    inputs = x
    labels = y
    logits = dynamicRNN(x, weights, biases)
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate = 0.1).minimize(loss)
    
    correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

75
(75, ?, 50)
(?, 50)
(50, 2)
(2,)


In [16]:
with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        feed_dict = {inputs: train_x, labels: train_y}
        _, l, predictions, acc = session.run([optimizer, loss, logits, accuracy],feed_dict = feed_dict)
        if (step % 100 == 0):
            print('Loss at step %d: %f acc: %f' % (step, l,acc))
    print("Testing Accuracy:", session.run(accuracy, feed_dict={inputs: test_x, labels: test_y}))

Initialized
Loss at step 0: 1.133565 acc: 0.495274
Loss at step 100: 0.693112 acc: 0.504726
Loss at step 200: 0.693103 acc: 0.504726
Loss at step 300: 0.693103 acc: 0.504726
Loss at step 400: 0.693102 acc: 0.504726
Loss at step 500: 0.693102 acc: 0.504726
Loss at step 600: 0.693102 acc: 0.504726


KeyboardInterrupt: 